In [304]:
using LinearAlgebra
using PlotlyJS

In [305]:
function householder(x)
    """Computes the Householder transformation for input vector x
    
    Returns 
    beta: float, the multiplier for future Householder reflection
    v: vector, the householder reflector vector  
    """
    sigma = dot(x[2:end],x[2:end])
    v = copy(x)

    if sigma == 0
        beta = 0
        return beta, v
    end

    sq = sqrt(x[1]^2 + sigma)
    if x[1] > 0
        v[1] += sq
    else
        v[1] -= sq
    end

    beta = 2.0 / (v[1]^2 + sigma)

    return beta, v
end


function apply_householder_col!(B, H, cidx, m)
    """Apply Householder reflection from the left (B<- H * B)
    
    Returns (inplace) 
    B: input matrix
    H: householder vectors v  
    """
    beta, v = householder(B[cidx:m, cidx])
    B[cidx:m, cidx:end] = B[cidx:m, cidx:end] - beta * v * (v' * B[cidx:m, cidx:end])
    H[cidx:end, cidx] = v / norm(v)

    return beta
end

function apply_householder_row!(B, H, ridx, n)
    """Apply Householder reflection from the right (B<- B * H)
    
    Returns (inplace) 
    B: input matrix
    H: householder vectors v  
    """
    beta, v = householder(B[ridx, ridx+1:n])
    B[ridx:end, ridx+1:n] = B[ridx:end, ridx+1:n] - (B[ridx:end, ridx+1:n] * v) * (beta * v')
    H[ridx, ridx+1:end] = v / norm(v)

end

function update_col_sim!(U, H, cidx, m)
    """Update column unitary transform (U<- H * U)
    
    Input:
    U: Left Householder matrix (orthogonal)
    H: householder vectors v
    cidx: index of column where householder transform is applied
    m: number of columns of original matrix
    
    Returns (inplace) 
    U: Left Householder matrix (orthogonal)
    """
    v = H[cidx:end, cidx]
    if cidx > 1
        v = [zeros(cidx-1);v]
    end
    
    
    
    U[:,:] = U - 2 * v * (v' * U)
    
    return U
    
end

function update_row_sim!(V, H, ridx, n)
    """Update row unitary transform (V<- V * H)
    
    Input:
    V: Right Householder matrix (orthogonal)
    H: householder vectors v
    ridx: index of row where householder transform is applied
    n: number of rows of original matrix
    
    Returns (inplace) 
    V: Right Householder matrix (orthogonal) 
    """
    v = H[ridx, ridx+1:end]
    v = [zeros(ridx);v]
    
    

    V[:,:] = V - 2 * (V * v) * v'
    
    return V
    
end


function bidiagonalize(A, return_orth = false)
    """Performs matrix bidiagonalization (for future singular value decomposition computation)
    
    Input:
    A: Rectangular m x n matrix
    
    Returns (inplace) 
    B: bidiagonal matrix with same singular values as input matrix (A = U * B * V')
    """
    m, n = size(A)
    B = copy(A)
    H = copy(A)  # v vectors in householder
    
    if return_orth
        U = zeros(m, m) + I
        V = zeros(n, n) + I
    end
    
    for k = 1:min(m,n)
        
        # column
        
        apply_householder_col!(B, H, k, m)
    
        
        if return_orth
            update_col_sim!(U, H, k, m)
        end
        
        
        # row
        if k < n
            apply_householder_row!(B, H, k, n) 
            
            if return_orth 
                update_row_sim!(V, H, k, n)
            end
        end
        
                
    end
    
    if return_orth
        V[:, end] = -V[:, end] 
        U[end, :] = -U[end, :]
        return U', B, V
    else
        return B, H
    end
end



bidiagonalize (generic function with 2 methods)

In [386]:
function givens_rotation(v)
    """ Computers the 2x2 Givens Rotation matrix for a given 2d vector v.
    
    Input:
    v: 2d vector
    
    Returns
    G:  givens rotation matrix
    """
    a, b = v[1], v[2]
    if b == 0
        c = 1
        s = 0
    else
        if abs(b) > abs(a)
            tau = -a/b
            s = 1.0/sqrt(1.0+tau*tau)
            c = s*tau
        else
            tau = -b/a
            c = 1.0/sqrt(1.0+tau*tau)
            s = c*tau
        end
    end
    return [c -s;s c]
end


function svd_golub_reinsh!(B, U, V, maxiter, eps=1e-12)
    """Computes the Singular value decomposition of a bidiagonal matrix. Based on 
    https://www.cs.utexas.edu/~inderjit/public_papers/HLA_SVD.pdf
    and
    https://link.springer.com/article/10.1007/s11075-022-01459-9
    
    Returns  
    U: Left singular vector matrix
    S: Singular values matrix
    Vt: Right singular vector matrix
    """
    
    function golub_kahan_step!(B, U, V, l, k)
        # Considering B22 -> (l:k x l:k)
        
        # Step 1: compute appropriate shift
        
        if k > 2
            a, b, d = B[k-1, k-1]^2 + B[k-2, k-1]^2, B[k-1, k-1] * B[k-1, k], B[k, k]^2 + B[k-1, k]^2
            t, r = - a - d, a * d - b^2
            s_1, s_2 = 0.5 * (-t + sqrt(t^2 -4 * r)), 0.5 * (-t - sqrt(t^2 -4 * r))
            if abs(B[k, k]^2 + B[k-1, k]^2 - s_1) < abs(B[k, k]^2 + B[k-1, k]^2 - s_2)
                s = s_1 
            else
                s = s_2 
            end
        else
           s = B[2, 2]^2 + B[1, 2]^2 
        end
        
        # Step 2: Computing Givens Rotations
        alpha, beta = B[l, l]^2-s, B[l, l] * B[l, l + 1]
        v = [alpha;beta]
        for p = l:k-1
            # Right rotation
            G = givens_rotation(v)
            B[:, p:p+1] = B[:, p:p+1] * G'
            V[:, p:p+1] = V[:, p:p+1] * G'
            
            
            # Left rotation
            alpha, beta = B[p, p], B[p + 1, p] 
            v = [alpha; beta]
            G = givens_rotation(v)
            B[p:p+1, :] = G * B[p:p+1, :]
            U[:, p:p+1] = U[:, p:p+1] * G'
            
            # Updating alpha, beta and v
            if p < k -1 
                alpha, beta = B[p, p+1], B[p, p+2]
                v = [alpha;beta]
            end
        end
    end
    
    m, n = size(B)
    maxiter_ = maxiter * log2(n * m) 
    
    for iters = 1:min(maxiter, maxiter_)
        
        # Step 1 of main loop
        # Find l and k such that matrix can be blocked as B11, B22 and B33 (diag) 
        
        k = n
        for j = n:-1:2  
            k = abs(B[j-1, j]) < eps * (abs(B[j, j]) + abs(B[j-1, j-1]))  ? j-1 : k
        end

        
        if k == 1
           # Diagonal matrix, convergence obtained
           return U, B, V
        end
        
        # Step 2 of main loop
        # Performing Golub Kahan at matrix B22
        if k == n 
            # B22 should be the full matrix 
            golub_kahan_step!(B, U, V, 1, n)
    
        else 
            # B33 (diagonal) found, have to check where B22 starts (l) and ends (k)
            l = k
            for j = k:-1:2
                l = abs(B[j-1, j]) < eps * (abs(B[j, j]) + abs(B[j-1, j-1])) ? j-1 : l
                
                if l!=k
                    break
                end
                
            end 
            
            # If l is equal to k after previous scan, then l = 1 and there is no B11 matrix
            l = (l == k) ? 1 : l 
            
            golub_kahan_step!(B, U, V, l, k)
        end
    end
end


function svd_num(A, maxiter = 80, save_results = false)
   """Performs singular value decompositon of a rectangular m x n matrix A.
    
    Returns 
        U: Left singular vector matrix
        S: Singular values matrix
        Vt: Right singular vector matrix
    """ 
    # Flag for transpose (case n > m)
    m, n = size(A)
    transpose_flag = false
    
    if n > m
        transpose_flag = true
    end
    
    # First main step: Find bidiagonalized matrix B
    if transpose_flag
        P_l, B, P_r = bidiagonalize(A', true);
    else
        P_l, B, P_r = bidiagonalize(A, true);
    end
    
    # Second main step: perform svd_golub_reinsh algorithm iterations
    if save_results
        t = min(m, n)
        results = zeros(2, t-1, maxiter)
        
        for k = 1:maxiter
            for j = 2:t
                results[:, j-1, k] = copy(B[j-1:j, j])
            end
            svd_golub_reinsh!(B, P_l, P_r, 1)
        end
        
        B = [i == j ? B[i, j] : 0 for i in 1:size(B, 1), j in 1:size(B, 2)];
        # Check if transpose
        if transpose_flag
            return P_r, B', P_l, results
        else
            return P_l, B, P_r, results
        end
        
    else
        svd_golub_reinsh!(B, P_l, P_r, maxiter)
        B = [i == j ? B[i, j] : 0 for i in 1:size(B, 1), j in 1:size(B, 2)];
        
        # Check if transpose
        if transpose_flag
            return P_r, B', P_l
        else
            return P_l, B, P_r
        end
    end
    
    
    
    
    
end

svd_num (generic function with 3 methods)

## Initial tests

In [387]:
# Example usage 1:
A = randn(5, 7)

5×7 Matrix{Float64}:
  1.29227     0.447238  -0.198909  …   1.16099   -1.4223      1.08897
 -1.91053     1.29579    0.383112     -0.377166   0.387062   -0.540492
  0.0890129   0.586602  -0.628978      0.851026  -0.119422    0.423432
  1.34847    -0.60477    2.01311      -0.988612  -0.0703459  -1.01091
  0.840133    0.130262  -0.3901       -1.02072    0.496854    0.206035

In [388]:
A

5×7 Matrix{Float64}:
  1.29227     0.447238  -0.198909  …   1.16099   -1.4223      1.08897
 -1.91053     1.29579    0.383112     -0.377166   0.387062   -0.540492
  0.0890129   0.586602  -0.628978      0.851026  -0.119422    0.423432
  1.34847    -0.60477    2.01311      -0.988612  -0.0703459  -1.01091
  0.840133    0.130262  -0.3901       -1.02072    0.496854    0.206035

In [389]:
P_l, B, P_r = bidiagonalize(A, true);

In [390]:
B

5×7 Matrix{Float64}:
 -2.80217      -1.58007  -1.11022e-16  …   2.22045e-16  -1.11022e-16
 -2.22045e-16   1.81411  -2.62289         -2.22045e-16   2.22045e-16
  1.38778e-17   0.0       2.53583          3.46945e-18  -5.55112e-17
  0.0           0.0       2.22045e-16      1.11022e-16   0.0
  1.11022e-16   0.0       4.44089e-16      1.02677       1.11022e-16

In [391]:
show(stdout, "text/plain", opnorm(P_l * B * P_r' - A, 1))

4.274358644806853e-15

In [392]:
U, S, V = svd_num(A);

In [393]:
show(stdout, "text/plain", U)

5×5 Matrix{Float64}:
  0.37717   -0.668243    0.0966841  -0.613449    -0.15977
  0.297924   0.71078     0.306664   -0.55618      0.051526
  0.670407  -0.102872    0.293492    0.436968     0.512732
 -0.474364  -0.171748    0.85896     0.00810081   0.0872012
 -0.307438  -0.0903532  -0.269555   -0.351193     0.837449

In [394]:
show(stdout, "text/plain", S)

5×7 adjoint(::Matrix{Real}) with eltype Real:
 -4.09424   0        0         0       0        0  0
  0        -3.17969  0         0       0        0  0
  0         0        2.40351   0       0        0  0
  0         0        0        -1.0263  0        0  0
  0         0        0         0       1.26891  0  0

In [395]:
show(stdout, "text/plain", V')

7×7 Matrix{Float64}:
  0.224722   -0.291831    0.297386    0.728677   -0.410047   0.151573  -0.231977
  0.798247   -0.205652   -0.0501401   0.0247874   0.273434  -0.378977   0.314628
  0.206778    0.0242101   0.727267   -0.52004    -0.136335  -0.103274  -0.357835
 -0.0239963   0.769139    0.207145    0.279038   -0.214261  -0.418965   0.2562
  0.442811    0.277743   -0.332662   -0.276925   -0.559207   0.469621   0.0785431
  0.187116    0.359564    0.272657    0.184812    0.591173   0.614393   0.0154618
  0.194615    0.271943   -0.390148    0.0962761   0.174362  -0.221769  -0.804427

In [396]:
show(stdout, "text/plain", opnorm(U * S * V' - A, 1)/opnorm(A, 1))

8.614346197595915e-16

In [397]:
show(stdout, "text/plain", opnorm(U * U' - I, 1))

3.2057663169902076e-15

In [398]:
show(stdout, "text/plain", opnorm(V * V' - I, 1))

1.5385204872580683e-15

In [399]:
# Example usage 2:
A = randn(9, 7)

9×7 Matrix{Float64}:
 -0.887124    0.179505   0.475723   …   0.373461  -1.87523    1.92313
 -1.31888    -1.1728    -0.613743      -0.152413   0.456912   1.62091
  2.53349     1.12529    0.4377         0.682256   0.192326   1.54823
  0.472543   -1.4034    -0.0483896     -1.4389     0.949135  -1.24798
  0.416243    0.992643   0.824781      -0.34751    2.18749   -0.0378228
 -0.219434    0.981766   1.13015    …  -1.84485   -0.486568   0.326385
  0.0953919   0.411669  -0.480796      -1.51698    0.840462  -0.127824
  0.837387    0.470189   0.92463        0.539794   0.515653   0.415429
  1.23483    -0.580178   0.915381       0.917184   0.312006  -0.513466

In [400]:
U, S, V = svd_num(A);

In [401]:
show(stdout, "text/plain", U)

9×9 adjoint(::Matrix{Float64}) with eltype Float64:
 -0.510114   -0.45394      0.20805    -0.0365872  -0.292803    -0.175781   0.273132   0.471798     0.274358
 -0.0017078  -0.363        0.0832691   0.78794     0.217238    -0.21611    0.182992  -0.243528    -0.231856
 -0.610093    0.472174     0.0735493   0.137582    0.0587656   -0.423643  -0.377847  -0.200249     0.121267
  0.522321    0.176395    -0.0350247   0.0392724   0.00669206  -0.649635   0.192789   0.132951     0.465169
  0.0270786   0.47851      0.32095     0.420176   -0.325982     0.45667    0.299028   0.00443733   0.294089
  0.0309906   0.00470819   0.729233   -0.390375    0.0737694   -0.170323   0.299261  -0.387968    -0.200788
  0.230869    0.129075     0.400386    0.158488   -0.157825    -0.129733  -0.40515    0.568077    -0.464773
 -0.154332    0.263088     0.0181863  -0.0236616   0.772467     0.104273   0.331681   0.431336    -0.049339
 -0.126235    0.305347    -0.384339   -0.0521669  -0.36091     -0.235874   0.508767 

In [402]:
show(stdout, "text/plain", S)

9×7 Matrix{Real}:
 4.38498   0         0         0        0       0        0
 0        -4.20818   0         0        0       0        0
 0         0        -3.03526   0        0       0        0
 0         0         0        -2.4451   0       0        0
 0         0         0         0       -1.1049  0        0
 0         0         0         0        0       1.76178  0
 0         0         0         0        0       0        1.5846
 0         0         0         0        0       0        0
 0         0         0         0        0       0        0

In [403]:
show(stdout, "text/plain", V')

7×7 adjoint(::Matrix{Float64}) with eltype Float64:
 -0.251467  -0.30926     -0.192893   -0.373978    -0.450161     0.331459   -0.592912
 -0.705499  -0.263134    -0.253238   -0.05449     -0.00128848  -0.553085    0.245007
  0.188517  -0.495013    -0.211879    0.0040914    0.738436    -0.066562   -0.353266
  0.183295   0.2915       0.279392   -0.246083    -0.0722774   -0.697791   -0.500677
 -0.144402  -0.00531045  -0.024474    0.888898    -0.17247     -0.0634296  -0.393208
 -0.52684    0.610428     0.0125195  -0.0800308    0.457089     0.275741   -0.241437
 -0.267372  -0.363946     0.880315    0.00731615   0.0903649    0.113282    0.00694201

In [404]:
show(stdout, "text/plain", opnorm(U * S * V' - A, 1)/opnorm(A, 1))

5.627428485739987e-14

In [405]:
show(stdout, "text/plain", opnorm(U * U' - I, 1))

3.712868460778858e-15

In [406]:
show(stdout, "text/plain", opnorm(V * V' - I, 1))

2.7876478511029994e-15

In [407]:
# Example usage 3:
A = randn(7, 7)
A = A + A'

7×7 Matrix{Float64}:
  1.18542    -0.432996  -0.843059  …  -0.0644463   2.46817    0.232591
 -0.432996   -2.09249    0.660181     -1.99029     2.78665   -1.19762
 -0.843059    0.660181   0.387442      1.54375    -0.293791  -1.65488
 -0.0782165   1.51003    0.992661     -0.899455   -3.23983    1.582
 -0.0644463  -1.99029    1.54375      -0.654      -0.159486   0.898734
  2.46817     2.78665   -0.293791  …  -0.159486    1.446      0.734863
  0.232591   -1.19762   -1.65488       0.898734    0.734863  -0.92576

In [408]:
U, S, V = svd_num(A);

In [409]:
show(stdout, "text/plain", U)

7×7 adjoint(::Matrix{Float64}) with eltype Float64:
  0.186806   -0.459279   -0.274878    -0.127755   -0.55663    0.593675  -5.26547e-5
  0.595898   -0.163218    0.493377    -0.378418    0.22971    0.04857   -0.420142
  0.0886797   0.201796    0.571979     0.361451    0.0452521  0.513294   0.477587
 -0.488437    0.408035    0.00613621  -0.523996    0.143258   0.493734  -0.23671
  0.0400427   0.0661217  -0.204459     0.643714    0.235017   0.302706  -0.625292
 -0.493778   -0.742063    0.211632     0.0323685   0.394542   0.056176   0.0295159
  0.343736   -0.0133849  -0.516946    -0.140869    0.635555   0.207751   0.384057

In [410]:
show(stdout, "text/plain", S)

7×7 Matrix{Real}:
 6.50138  0         0         0        0          0          0
 0        5.47536   0         0        0          0          0
 0        0        -2.20783   0        0          0          0
 0        0         0        -1.92312  0          0          0
 0        0         0         0        0.535034   0          0
 0        0         0         0        0         -0.638111   0
 0        0         0         0        0          0         -4.05537

In [411]:
show(stdout, "text/plain", V')

7×7 adjoint(::Matrix{Float64}) with eltype Float64:
 -0.186806    -0.595898  -0.0886797   0.488437    -0.0400427   0.493778   -0.343736
 -0.459279    -0.163218   0.201796    0.408035     0.0661217  -0.742063   -0.0133849
  0.274878    -0.493377  -0.571979   -0.00613621   0.204459   -0.211632    0.516946
  0.127755     0.378418  -0.361451    0.523996    -0.643714   -0.0323685   0.140869
  0.55663     -0.22971   -0.0452521  -0.143258    -0.235017   -0.394542   -0.635555
 -0.593675    -0.04857   -0.513294   -0.493734    -0.302706   -0.056176   -0.207751
 -5.26547e-5  -0.420142   0.477587   -0.23671     -0.625292    0.0295159   0.384057

In [412]:
show(stdout, "text/plain", opnorm(U * S * V' - A, 1)/opnorm(A, 1))

5.507826889774333e-14

In [413]:
show(stdout, "text/plain", opnorm(U * U' - I, 1))

2.631518128901807e-15

In [414]:
show(stdout, "text/plain", opnorm(V * V' - I, 1))

1.9601398518470555e-15

## Cubic convergence depiction

### Square 7 x 7

In [415]:
A = randn(7, 7)

7×7 Matrix{Float64}:
  0.216951   -0.614549   1.08938    1.23056    0.307265  -0.729862   1.29003
 -1.16573     0.700209   1.16469    0.510615   0.867353   1.63862   -0.328677
  0.0591166  -0.814118   2.64423   -2.11629    1.29781    0.902478   0.560716
 -1.16342     0.60594    0.817585  -0.234041   0.172582  -1.28458    0.690064
 -0.223629   -0.585731   0.322173  -0.372459  -1.06023    1.04343    1.03257
 -2.11292     1.64245    0.740388   0.271954   1.17428    1.36481    1.66583
  0.542812   -0.908377  -0.143009   0.16448   -0.449775  -1.14218   -0.803935

In [416]:
U, S, V, results = svd_num(A, 20, true);

In [417]:
show(stdout, "text/plain", opnorm(U * S * V' - A, 1)/opnorm(A, 1))

4.6204672046101144e-14

In [418]:
function plot_convergence_error(error_vectors, labels)
    iterations = 1:length(error_vectors[1])
    traces = Vector{GenericTrace{Dict{Symbol, Any}}}()
    
    for (i, error_vector) in enumerate(error_vectors)
        trace = scatter(
            x = iterations,
            y = error_vector,
            mode = "lines+markers",
            marker_color = i,
            line_shape = "linear",
            name = labels[i]
        )
        push!(traces, trace)
    end
    
    layout = Layout(
        title = "Value of element above diagonal σ_i",
        xaxis_title = "Iteration",
        yaxis_title = "Error",
        yaxis_type = "log",
        yaxis_tickformat = "e",
        showlegend = true
    )
    
    plot(traces, layout)
end

error_vectors = [
    abs.(results[1, 1, :]),
    abs.(results[1, 2, :]),
    abs.(results[1, 3, :]),
    abs.(results[1, 4, :]),
    abs.(results[1, 5, :]),
    abs.(results[1, 6, :])
]
labels = ["σ2", "σ3", "σ4", "σ5", "σ6", "σ7"]
plot_convergence_error(error_vectors, labels)

data: [
  "scatter with fields line, marker, mode, name, type, x, and y",
  "scatter with fields line, marker, mode, name, type, x, and y",
  "scatter with fields line, marker, mode, name, type, x, and y",
  "scatter with fields line, marker, mode, name, type, x, and y",
  "scatter with fields line, marker, mode, name, type, x, and y",
  "scatter with fields line, marker, mode, name, type, x, and y"
]

layout: "layout with fields margin, showlegend, template, title, xaxis, and yaxis"

In [419]:
# σ_2
show(stdout, "text/plain", results[2, 1, :]')

1×20 adjoint(::Vector{Float64}) with eltype Float64:
 2.28433  2.60322  -3.34888  -3.46567  -3.47521  -3.47793  -3.47895  -3.47895  -3.47895  -3.47895  -3.47895  -3.47895  -3.47895  -3.47895  -3.47895  -3.47895  -3.47895  -3.47895  -3.47895  -3.47895

In [420]:
# σ_3
show(stdout, "text/plain", results[2, 2, :]')

1×20 adjoint(::Vector{Float64}) with eltype Float64:
 2.33098  0.890415  -0.555888  0.502768  0.407899  -0.345732  -0.994724  -2.43166  -2.64887  -2.6565  -2.6565  -2.6565  -2.6565  -2.6565  -2.6565  -2.6565  -2.6565  -2.6565  -2.6565  -2.6565

In [421]:
# σ_4
show(stdout, "text/plain", results[2, 3, :]')

1×20 adjoint(::Vector{Float64}) with eltype Float64:
 -1.31927  1.59928  1.87086  1.77673  1.94616  -2.11325  -0.682696  -0.279246  -0.256348  -0.255611  -0.255611  -0.255611  -0.255611  -0.255611  -0.255611  -0.255611  -0.255611  -0.255611  -0.255611  -0.255611

In [422]:
# σ_5
show(stdout, "text/plain", results[2, 4, :]')

1×20 adjoint(::Vector{Float64}) with eltype Float64:
 1.3314  0.812126  -0.735281  0.803759  0.895994  0.972659  1.04613  1.04623  1.04623  1.04623  1.04623  1.04623  1.04623  1.04623  1.04623  1.04623  1.04623  1.04623  1.04623  1.04623

In [423]:
# σ_6
show(stdout, "text/plain", results[2, 5, :]')

1×20 adjoint(::Vector{Float64}) with eltype Float64:
 1.00129  -1.75554  1.98063  2.02854  2.04046  2.04046  2.04046  2.04046  2.04046  2.04046  2.04046  2.04046  2.04046  2.04046  2.04046  2.04046  2.04046  2.04046  2.04046  2.04046

In [424]:
# σ_7
show(stdout, "text/plain", results[2, 6, :]')

1×20 adjoint(::Vector{Float64}) with eltype Float64:
 -1.735  1.75875  1.75767  1.75767  1.75767  1.75767  1.75767  1.75767  1.75767  1.75767  1.75767  1.75767  1.75767  1.75767  1.75767  1.75767  1.75767  1.75767  1.75767  1.75767

In [425]:
S

7×7 Matrix{Real}:
 5.04401   0         0        0         0        0        0
 0        -3.47895   0        0         0        0        0
 0         0        -2.6565   0         0        0        0
 0         0         0       -0.255611  0        0        0
 0         0         0        0         1.04623  0        0
 0         0         0        0         0        2.04046  0
 0         0         0        0         0        0        1.75767

### Rectangular 9 x 7

In [426]:
A = randn(9, 7)

9×7 Matrix{Float64}:
  0.531472   -0.471773    0.465741  …   0.351069   -1.99085   -1.03265
  0.0360165   0.0647897  -1.15668      -0.717835   -1.29214   -0.366458
 -0.755366   -0.776155   -1.18559       0.527637   -0.101869   0.364256
  0.545841   -1.29451     1.19833       0.917372   -1.81745    1.26737
 -0.0999943   0.236245   -0.19643       0.561593   -0.251219  -0.835842
  0.377985   -2.30091     0.282727  …   0.510967   -1.5463     1.43837
 -1.19762    -0.31658    -1.15287      -1.46589    -0.615143   0.990382
  1.44659    -1.52206     1.07478      -1.95508    -0.174399   0.115243
 -0.363244    0.438131   -0.753763      0.0778388   0.500505  -0.10534

In [427]:
U, S, V, results = svd_num(A, 20, true);

In [428]:
show(stdout, "text/plain", opnorm(U * S * V' - A, 1)/opnorm(A, 1))

4.358432614134915e-14

In [429]:
error_vectors = [
    abs.(results[1, 1, :]),
    abs.(results[1, 2, :]),
    abs.(results[1, 3, :]),
    abs.(results[1, 4, :]),
    abs.(results[1, 5, :]),
    abs.(results[1, 6, :])
]
labels = ["σ2", "σ3", "σ4", "σ5", "σ6", "σ7"]
plot_convergence_error(error_vectors, labels)

data: [
  "scatter with fields line, marker, mode, name, type, x, and y",
  "scatter with fields line, marker, mode, name, type, x, and y",
  "scatter with fields line, marker, mode, name, type, x, and y",
  "scatter with fields line, marker, mode, name, type, x, and y",
  "scatter with fields line, marker, mode, name, type, x, and y",
  "scatter with fields line, marker, mode, name, type, x, and y"
]

layout: "layout with fields margin, showlegend, template, title, xaxis, and yaxis"

In [430]:
# σ_2
show(stdout, "text/plain", results[2, 1, :]')

1×20 adjoint(::Vector{Float64}) with eltype Float64:
 2.72406  -3.70802  -3.44529  -3.47032  -3.54304  -3.59026  -3.61786  -3.63253  -3.63968  -3.64711  -3.64727  -3.64728  -3.64728  -3.64728  -3.64728  -3.64728  -3.64728  -3.64728  -3.64728  -3.64728

In [431]:
# σ_3
show(stdout, "text/plain", results[2, 2, :]')

1×20 adjoint(::Vector{Float64}) with eltype Float64:
 -1.96495  1.85553  -3.23894  -3.24737  -3.17418  -3.13219  -3.10919  -3.09762  -3.09237  -3.09164  -3.0915  -3.09149  -3.09149  -3.09149  -3.09149  -3.09149  -3.09149  -3.09149  -3.09149  -3.09149

In [432]:
# σ_4
show(stdout, "text/plain", results[2, 3, :]')

1×20 adjoint(::Vector{Float64}) with eltype Float64:
 -2.96145  1.0178  -0.575386  1.416  -2.75568  -2.98753  -2.98785  -2.98683  -2.98601  -2.98064  -2.98064  -2.98064  -2.98064  -2.98064  -2.98064  -2.98064  -2.98064  -2.98064  -2.98064  -2.98064

In [433]:
# σ_5
show(stdout, "text/plain", results[2, 4, :]')

1×20 adjoint(::Vector{Float64}) with eltype Float64:
 -0.766017  1.92396  -2.36424  0.879424  -0.43919  -0.39838  -0.398174  -0.398173  -0.398173  -0.398173  -0.398173  -0.398173  -0.398173  -0.398173  -0.398173  -0.398173  -0.398173  -0.398173  -0.398173  -0.398173

In [434]:
# σ_6
show(stdout, "text/plain", results[2, 5, :]')

1×20 adjoint(::Vector{Float64}) with eltype Float64:
 2.4361  -1.06266  0.809062  0.852169  0.875181  0.889486  0.88949  0.88949  0.88949  0.88949  0.88949  0.88949  0.88949  0.88949  0.88949  0.88949  0.88949  0.88949  0.88949  0.88949

In [435]:
# σ_7
show(stdout, "text/plain", results[2, 6, :]')

1×20 adjoint(::Vector{Float64}) with eltype Float64:
 -1.26356  -1.31652  -1.32047  -1.32047  -1.32047  -1.32047  -1.32047  -1.32047  -1.32047  -1.32047  -1.32047  -1.32047  -1.32047  -1.32047  -1.32047  -1.32047  -1.32047  -1.32047  -1.32047  -1.32047

In [436]:
S

9×7 Matrix{Real}:
 5.29825   0         0         0         0         0         0
 0        -3.64728   0         0         0         0         0
 0         0        -3.09149   0         0         0         0
 0         0         0        -2.98064   0         0         0
 0         0         0         0        -0.398173  0         0
 0         0         0         0         0         0.88949   0
 0         0         0         0         0         0        -1.32047
 0         0         0         0         0         0         0
 0         0         0         0         0         0         0

### Rectangular 7 x 9

In [441]:
A = randn(7, 9)

7×9 Matrix{Float64}:
 -0.336277  -0.432672   -0.134245  …   0.749773   1.46903     2.10639
 -0.335129  -0.801359   -0.699933     -1.18924    0.436244    1.47546
 -0.599446   0.42317    -0.876958     -1.53689    1.74084    -0.280173
  0.657268  -0.0463785  -0.159689      1.92622   -0.0598371  -1.70997
  1.53532    1.30325     0.850459     -1.26265    1.15781     0.840584
 -0.889243  -1.13256    -0.447852  …  -1.79341   -1.61142     2.06114
 -0.152212   0.470221    0.362391      1.24283   -1.24057     0.462675

In [442]:
U, S, V, results = svd_num(A, 20, true);

In [443]:
show(stdout, "text/plain", opnorm(U * S * V' - A, 1)/opnorm(A, 1))

3.2301829010138344e-15

In [444]:
error_vectors = [
    abs.(results[1, 1, :]),
    abs.(results[1, 2, :]),
    abs.(results[1, 3, :]),
    abs.(results[1, 4, :]),
    abs.(results[1, 5, :]),
    abs.(results[1, 6, :])
]
labels = ["σ2", "σ3", "σ4", "σ5", "σ6", "σ7"]
plot_convergence_error(error_vectors, labels)

data: [
  "scatter with fields line, marker, mode, name, type, x, and y",
  "scatter with fields line, marker, mode, name, type, x, and y",
  "scatter with fields line, marker, mode, name, type, x, and y",
  "scatter with fields line, marker, mode, name, type, x, and y",
  "scatter with fields line, marker, mode, name, type, x, and y",
  "scatter with fields line, marker, mode, name, type, x, and y"
]

layout: "layout with fields margin, showlegend, template, title, xaxis, and yaxis"

In [445]:
# σ_2
show(stdout, "text/plain", results[2, 1, :]')

1×20 adjoint(::Vector{Float64}) with eltype Float64:
 -3.65831  -3.43602  -3.64846  3.98731  -4.1954  -4.28319  -4.42918  -4.43429  -4.43505  -4.43514  -4.43515  -4.4351  -4.43507  -4.43504  -4.43504  -4.43504  -4.43504  -4.43504  -4.43504  -4.43504

In [446]:
# σ_3
show(stdout, "text/plain", results[2, 2, :]')

1×20 adjoint(::Vector{Float64}) with eltype Float64:
 -2.73866  3.99237  3.71717  -3.2367  2.91558  2.57934  -1.84467  -1.74927  -1.74192  -1.74096  -1.7408  -1.74079  -1.74079  -1.74079  -1.74079  -1.74079  -1.74079  -1.74079  -1.74079  -1.74079

In [447]:
# σ_4
show(stdout, "text/plain", results[2, 3, :]')

1×20 adjoint(::Vector{Float64}) with eltype Float64:
 3.97082  -2.11557  -2.45113  -2.24982  -2.09865  -2.10476  -2.63348  2.59578  2.60453  2.60573  2.60589  2.60589  2.60589  2.60589  2.60589  2.60589  2.60589  2.60589  2.60589  2.60589

In [448]:
# σ_5
show(stdout, "text/plain", results[2, 4, :]')

1×20 adjoint(::Vector{Float64}) with eltype Float64:
 2.66584  2.46244  2.28655  -2.07992  -2.32942  -2.56389  2.76874  -2.95772  -2.95925  -2.95925  -2.95925  -2.95925  -2.95925  -2.95925  -2.95925  -2.95925  -2.95925  -2.95925  -2.95925  -2.95925

In [449]:
# σ_6
show(stdout, "text/plain", results[2, 5, :]')

1×20 adjoint(::Vector{Float64}) with eltype Float64:
 3.49228  3.50474  2.93016  -3.57033  -3.57062  -3.57062  -3.57062  -3.57062  -3.57062  -3.57062  -3.57062  -3.57062  -3.57062  -3.57062  -3.57062  -3.57062  -3.57062  -3.57062  -3.57062  -3.57062

In [450]:
# σ_7
show(stdout, "text/plain", results[2, 6, :]')

1×20 adjoint(::Vector{Float64}) with eltype Float64:
 1.32663  1.30208  1.30208  1.30208  1.30208  1.30208  1.30208  1.30208  1.30208  1.30208  1.30208  1.30208  1.30208  1.30208  1.30208  1.30208  1.30208  1.30208  1.30208  1.30208

In [451]:
S

7×9 adjoint(::Matrix{Real}) with eltype Real:
 5.00606   0         0        0         0         0        0        0  0
 0        -4.43504   0        0         0         0        0        0  0
 0         0        -1.74079  0         0         0        0        0  0
 0         0         0        2.60589   0         0        0        0  0
 0         0         0        0        -2.95925   0        0        0  0
 0         0         0        0         0        -3.57062  0        0  0
 0         0         0        0         0         0        1.30208  0  0

### Badly conditioned square matrix

In [489]:
u = randn(7, 1)
u = u/norm(u)
A = (I - u * u') + 1e-9 * randn(7, 7)

7×7 Matrix{Float64}:
  0.850642     -0.0734124     0.000588389  …  -0.192072     0.117026
 -0.0734124     0.963916      0.000289207     -0.0944074    0.0575208
  0.000588389   0.000289206   0.999998         0.00075666  -0.000461021
  0.236744      0.116364     -0.000932643      0.304449    -0.185496
  0.12258       0.0602505    -0.0004829        0.157636    -0.0960451
 -0.192072     -0.0944074     0.00075666   …   0.752998     0.150494
  0.117026      0.0575208    -0.000461021      0.150494     0.908306

In [490]:
U, S, V, results = svd_num(A, 25, true);

In [491]:
show(stdout, "text/plain", opnorm(U * S * V' - A, 1)/opnorm(A, 1))

1.6144162962245394e-9

In [492]:
error_vectors = [
    abs.(results[1, 1, :]),
    abs.(results[1, 2, :]),
    abs.(results[1, 3, :]),
    abs.(results[1, 4, :]),
    abs.(results[1, 5, :]),
    abs.(results[1, 6, :])
]
labels = ["σ2", "σ3", "σ4", "σ5", "σ6", "σ7"]
plot_convergence_error(error_vectors, labels)

data: [
  "scatter with fields line, marker, mode, name, type, x, and y",
  "scatter with fields line, marker, mode, name, type, x, and y",
  "scatter with fields line, marker, mode, name, type, x, and y",
  "scatter with fields line, marker, mode, name, type, x, and y",
  "scatter with fields line, marker, mode, name, type, x, and y",
  "scatter with fields line, marker, mode, name, type, x, and y"
]

layout: "layout with fields margin, showlegend, template, title, xaxis, and yaxis"

In [493]:
# σ_2
show(stdout, "text/plain", results[2, 1, :]')

1×25 adjoint(::Vector{Float64}) with eltype Float64:
 -6.23717e-9  -0.0140052  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0

In [494]:
# σ_3
show(stdout, "text/plain", results[2, 2, :]')

1×25 adjoint(::Vector{Float64}) with eltype Float64:
 -0.0195109  1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0

In [495]:
# σ_4
show(stdout, "text/plain", results[2, 3, :]')

1×25 adjoint(::Vector{Float64}) with eltype Float64:
 1.0  -1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0

In [496]:
# σ_5
show(stdout, "text/plain", results[2, 4, :]')

1×25 adjoint(::Vector{Float64}) with eltype Float64:
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0

In [497]:
# σ_6
show(stdout, "text/plain", results[2, 5, :]')

1×25 adjoint(::Vector{Float64}) with eltype Float64:
 -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0

In [498]:
# σ_7
show(stdout, "text/plain", results[2, 6, :]')

1×25 adjoint(::Vector{Float64}) with eltype Float64:
 -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0

In [499]:
diag(S)

7-element Vector{Real}:
 -1.1223757029617512e-10
 -1.0000000021024165
 -1.0000000015716395
  1.0000000013928907
  0.9999999994387172
 -1.000000000319625
 -0.999999998608909

In [500]:
U, S, V = svd(A);

In [501]:
show(stdout, "text/plain", opnorm(U * diagm(S) * V' - A, 1)/opnorm(A, 1))

1.6625896668467898e-15

In [502]:
S

7-element Vector{Float64}:
 1.000000002576357
 1.0000000020201911
 1.000000000700176
 1.000000000372829
 0.999999999182472
 0.9999999985821685
 1.1223759622003846e-10

In [381]:
# 4 times
# bidiagonalize
# bidiagonalize (with matrices)
# from bidiag to diag (Golub Kahan) without singular vectors
# from bidiag to diag (Golub Kahan) with singular vectors